In [12]:
import torch
from torch import nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt
from datetime import datetime
import pickle

class TimeSeriesDataset(Dataset):
    def __init__(self, dataframe, seq_len=7*24, pred_len=24):
        self.seq_len = seq_len
        self.pred_len = pred_len
        self.scaler = MinMaxScaler()

        self.dataframe = self._preprocess(dataframe)

    def _preprocess(self, df):
        # If there are any missing values, fill them with the previous value in time-series
        df.fillna(method='ffill', inplace=True)

        # Normalize numerical columns to range [0, 1]
        numerical_cols = df.select_dtypes(include=[np.number]).columns
        df[numerical_cols] = self.scaler.fit_transform(df[numerical_cols])

        # One-hot encode categorical variables
        categorical_cols = df.select_dtypes(include=['object']).columns
        if not categorical_cols.empty:
            encoder = OneHotEncoder()
            encoded = encoder.fit_transform(df[categorical_cols])
            encoded_df = pd.DataFrame(encoded.toarray(), columns=encoder.get_feature_names(categorical_cols))
            
            # Drop original categorical columns and merge with encoded ones
            df.drop(columns=categorical_cols, inplace=True)
            df = pd.concat([df, encoded_df], axis=1)
        
        return df

    def __len__(self):
        return len(self.dataframe) - self.seq_len - self.pred_len + 1

    def __getitem__(self, idx):
        x = self.dataframe.iloc[idx:idx+self.seq_len]
        y = self.dataframe.iloc[idx+self.seq_len:idx+self.seq_len+self.pred_len, -56:] # Assuming last 56 columns are power values
        return torch.Tensor(x.values), torch.Tensor(y.values).reshape(-1)  # flatten y values
    
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device) # Initialize hidden state
        c0 = torch.zeros(self.n_layers, x.size(0), self.hidden_dim).to(x.device) # Initialize cell state

        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out


In [13]:
# Load the excel file
df = pd.read_excel('/home/kimyirum/EMS/ict-2023-ems/load/data/merged_data.xlsx')

# Initialize our dataset class
dataset = TimeSeriesDataset(df)

# Define the split sizes for train, validation, and test sets
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Split dataset
train_set, val_set, test_set = random_split(dataset, [train_size, val_size, test_size])

In [14]:
########################################## Hyperparameters ##########################################
input_dim = len(train_set[0][0][0]) # 63
output_dim = 24*56
hidden_dim = 256
n_layers = 7
learning_rate = 0.001
num_epochs = 200
batch_size = 128
use_thread = True
patience = 20 # number of epochs to wait before stopping
########################################################################################################

results_folder = "/home/kimyirum/EMS/ict-2023-ems/load/results/"
now = datetime.now()
now_str = now.strftime('%Y%m%d_%H%M%S')
filename_metrics = f'{now_str}.pkl'
filename_model = f'model_{now_str}.pt'

# DataLoader
if use_thread:
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=8)
    val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=8)
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=8)
else:
    train_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False)

# Initialize model, loss, and optimizer
model = LSTMModel(input_dim, hidden_dim, output_dim, n_layers)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_val_loss = float("inf") # initially set to infinity
no_improve_epoch = 0

########################################## Training loop ##########################################
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Validate the model
    model.eval()
    with torch.no_grad():
        val_losses = []
        for data, targets in val_loader:
            outputs = model(data)
            loss = criterion(outputs, targets)
            val_losses.append(loss.item())
        avg_val_loss = sum(val_losses) / len(val_losses)
        print(f'Epoch [{epoch+1}/{num_epochs}], Validation Loss: {avg_val_loss:.4f}', end='')
        
        # save model if validation loss has decreased
        if avg_val_loss < best_val_loss:
            print(", Save model")
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), results_folder+filename_model)
            no_improve_epoch = 0
        else:
            print("")
            no_improve_epoch += 1
            
        # early stopping
        if no_improve_epoch > patience:
            print('Early stopping...')
            break
########################################################################################################

########################################## Evaluate the model ##########################################
model.eval()
with torch.no_grad():
    all_targets = []
    all_outputs = []
    for data, targets in test_loader:
        outputs = model(data)
        all_targets.append(targets.numpy())
        all_outputs.append(outputs.numpy())

# Flatten targets and outputs to calculate metrics
all_targets = np.concatenate(all_targets).flatten()
all_outputs = np.concatenate(all_outputs).flatten()

# Calculate MAE, MSE and RMSE
mae = mean_absolute_error(all_targets, all_outputs)
mse = mean_squared_error(all_targets, all_outputs)
rmse = sqrt(mse)

# THIS IS NORMALIZATION VALUE
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}')
########################################################################################################


Epoch [1/200], Validation Loss: 0.0906, Save model
Epoch [2/200], Validation Loss: 0.0855, Save model
Epoch [3/200], Validation Loss: 0.2604
Epoch [4/200], Validation Loss: 0.0602, Save model
Epoch [5/200], Validation Loss: 0.0512, Save model
Epoch [6/200], Validation Loss: 0.0544
Epoch [7/200], Validation Loss: 0.0452, Save model
Epoch [8/200], Validation Loss: 0.0408, Save model
Epoch [9/200], Validation Loss: 0.0333, Save model
Epoch [10/200], Validation Loss: 0.0300, Save model
Epoch [11/200], Validation Loss: 0.0292, Save model
Epoch [12/200], Validation Loss: 0.0277, Save model
Epoch [13/200], Validation Loss: 0.0272, Save model
Epoch [14/200], Validation Loss: 0.0237, Save model
Epoch [15/200], Validation Loss: 0.0217, Save model
Epoch [16/200], Validation Loss: 0.0205, Save model
Epoch [17/200], Validation Loss: 0.0191, Save model
Epoch [18/200], Validation Loss: 0.0190, Save model
Epoch [19/200], Validation Loss: 0.0172, Save model
Epoch [20/200], Validation Loss: 0.0166, Save

In [17]:
# Save hyperparams
hyperparams = {
    'learning_rate': learning_rate,
    'batch_size': batch_size,
    'num_epochs': num_epochs,
    'hidden_dim': hidden_dim,
    'n_layers': n_layers
}

scalers = dataset.scaler

metrics = {
    'MAE': mae,
    'MSE': mse,
    'RMSE': rmse,
}

# Combine all results in one dictionary
results = {
    'Hyperparameters': hyperparams,
    'Scalers': scalers,
    'Metrics': metrics
}

# Save results to a pickle file
with open(results_folder + filename_metrics, 'wb') as f:
    pickle.dump(results, f)


# load test
# Load results from a pickle file
with open(results_folder + filename_metrics, 'rb') as f:
    loaded_results = pickle.load(f)

# Now you can access your data from the loaded_results dictionary
print(loaded_results['Hyperparameters'])
print(loaded_results['Scalers'])
print(loaded_results['Metrics'])


{'learning_rate': 0.001, 'batch_size': 128, 'num_epochs': 200, 'hidden_dim': 256, 'n_layers': 7}
MinMaxScaler()
{'MAE': 0.030826006, 'MSE': 0.0056500575, 'RMSE': 0.07516686409174889}


In [18]:
# Assuming df is the original dataset and it includes a 'date' column

# Get the building names
building_names = df.columns[-56:]  # adjust this as necessary

# DataLoader for test set
test_loader = DataLoader(dataset=test_set, batch_size=1, shuffle=False)

# Get the first sequence and its target from the test set
real_sequence, real_target = next(iter(test_loader))

# Switch model to eval mode
model.eval()

# Make prediction
with torch.no_grad():
    prediction = model(real_sequence)

print(prediction.shape, real_target.shape)
prediction = prediction.squeeze(0).reshape(24, 56).numpy()
real_target = real_target.view(24, 56).numpy()

padding = np.zeros((prediction.shape[0], 7))
prediction_pad = np.hstack((padding, prediction))
real_target_pad = np.hstack((padding, real_target))
# print(prediction_pad.shape, real_target_pad.shape)

# Apply inverse transformation
prediction_inv = dataset.scaler.inverse_transform(prediction_pad)
real_target_inv = dataset.scaler.inverse_transform(real_target_pad)

# Delete the first 7 columns
prediction_inv = np.delete(prediction_inv, np.s_[:7], axis=1)
real_target_inv = np.delete(real_target_inv, np.s_[:7], axis=1)

# Reshape them back to the original shape
prediction = prediction_inv.reshape(prediction.shape)
real_target = real_target_inv.reshape(real_target.shape)


# Calculate error (difference between real target and prediction)
error = real_target - prediction

# Create DataFrame for prediction
predicted_df = pd.DataFrame(prediction, columns=building_names)
real_target_df = pd.DataFrame(real_target, columns=building_names)
error_df = pd.DataFrame(error, columns=building_names)

# print("Predicted Values for next 24 hours:")
# print(predicted_df)

# print("Real Values for next 24 hours:")
# print(real_target_df)

print("Error for next 24 hours:")
print(error_df)

mae = mean_absolute_error(real_target, prediction)
mse = mean_squared_error(real_target, prediction)
rmse = sqrt(mse)
print(f'MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f} (no normalization)')


torch.Size([1, 1344]) torch.Size([1, 1344])
Error for next 24 hours:
        0_SV-2      1_SV-5    2_SV-6      3_SV-7  4_HV-NM1    5_HV-NM2  \
0    91.348845  108.863337 -0.002062   96.735128 -0.004243   74.124384   
1   122.130549   97.539897 -0.007082   97.797671  0.002816   85.080483   
2   113.191709  113.795971 -0.001610  119.980631  0.002716   77.136793   
3   103.615503  111.160176 -0.004209   85.685015 -0.006105   70.950913   
4   114.848384  113.821156  0.001948  112.603048 -0.004016   72.325043   
5   118.419181  126.947380  0.007613  114.388506 -0.002228   94.608645   
6   109.521006  120.083060  0.003595  111.769644 -0.003589   86.763988   
7   112.827925  106.986725  0.001788  108.196643  0.001706  109.870543   
8   117.021918  129.018566  0.004100  109.339179  0.003622   86.395682   
9   153.445605  142.908081 -0.009855  109.438649 -0.003228   85.875728   
10  138.931644  139.705652 -0.000105  123.446359  0.010569  104.429736   
11  141.491229  130.380165 -0.000672  141.3